In [1]:
import os
import json
import torch
import faiss
import numpy as np
import gc
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModel

c:\Users\thain\anaconda3\envs\ragbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# --------------------
# 0. Setup và Load model
# --------------------
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "intfloat/multilingual-e5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model.eval().to(device)

c:\Users\thain\anaconda3\envs\ragbot\lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


XLMRobertaModel(
  (embeddings): XLMRobertaEmbeddings(
    (word_embeddings): Embedding(250002, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): XLMRobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x XLMRobertaLayer(
        (attention): XLMRobertaAttention(
          (self): XLMRobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): XLMRobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=Tru

In [ ]:
# --------------------
# 1. Dataset + Embedding function
# --------------------
class TextDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx]

def embed_texts(texts, batch_size=2):
    embeddings = []
    dataset = TextDataset(texts)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    for batch in tqdm(dataloader, desc="Embedding batches"):
        try:
            encoded = tokenizer(
                [f"passage: {text}" for text in batch],
                return_tensors="pt", padding=True, truncation=True, max_length=512
            )
            encoded = {k: v.to(device) for k, v in encoded.items()}

            with torch.no_grad():
                outputs = model(**encoded)
                emb = outputs.last_hidden_state[:, 0, :] # First token (CLS) embedding
                emb = torch.nn.functional.normalize(emb, p=2, dim=1) # Normalize vector L2
                embeddings.append(emb.cpu().numpy())

            del encoded, outputs, emb
            gc.collect() # Clear cache
            torch.cuda.empty_cache() # Clear cache

        except RuntimeError as e:
            print(f"[!] Skipped batch due to CUDA error: {e}")
            gc.collect()
            torch.cuda.empty_cache()
            continue

    return np.vstack(embeddings)

In [4]:
# --------------------
# 2. Xử lý từng file .jsonl với khả năng resume
# --------------------
data_dir = "wiki_chunks"
output_dir = "wiki_embeddings"
os.makedirs(output_dir, exist_ok=True)

all_metadata = []
embedding_parts = []

# Lưu danh sách file đã xử lý
processed_files = set()
for fname in os.listdir(output_dir):
    if fname.startswith("embeddings_part_") and fname.endswith(".npy"):
        part_id = fname.split("_")[-1].split(".")[0]
        processed_files.add(int(part_id))

files = sorted(os.listdir(data_dir))  # Đảm bảo cùng thứ tự mỗi lần chạy

for idx, file in enumerate(tqdm(files, desc="Processing files")):
    if not file.endswith(".jsonl") or idx in processed_files:
        continue

    file_path = os.path.join(data_dir, file)
    texts = []
    metadata = []

    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            item = json.loads(line)
            texts.append(item["text"])
            metadata.append(item["metadata"])

    if not texts:
        continue

    print(f"[→] Embedding file: {file} (idx={idx})")

    embeddings = embed_texts(texts, batch_size=8)

    emb_file = os.path.join(output_dir, f"embeddings_part_{idx}.npy")
    meta_file = os.path.join(output_dir, f"metadata_part_{idx}.json")

    np.save(emb_file, embeddings)
    with open(meta_file, "w", encoding="utf-8") as f:
        json.dump(metadata, f, ensure_ascii=False, indent=2)

    embedding_parts.append(emb_file)

# Giới hạn chiều cao output
from IPython.display import display, HTML

display(HTML('''
<style>
.output_scroll {
    max-height: 300px;
    overflow-y: scroll;
}
</style>
'''))

Processing files:   0%|          | 0/762 [00:00<?, ?it/s]

[→] Embedding file: An Nam Cộng sản Đảng.jsonl (idx=0)


Processing files:   0%|          | 1/762 [00:00<06:30,  1.95it/s]

[→] Embedding file: Ban Chấp hành Trung ương Đảng Cộng sản Việt Nam.jsonl (idx=1)


Processing files:   0%|          | 2/762 [00:00<05:40,  2.23it/s]

...

Processing files: 100%|█████████▉| 760/762 [34:24<00:03,  1.61s/it]

[→] Embedding file: Ủy viên Trung ương Đảng Cộng sản Việt Nam.jsonl (idx=760)


Processing files: 100%|█████████▉| 761/762 [34:24<00:01,  1.16s/it]

[→] Embedding file: Ủy viên Đảng ủy Công an Trung ương Việt Nam.jsonl (idx=761)


Processing files: 100%|██████████| 762/762 [34:24<00:00,  2.71s/it]


In [5]:
# --------------------
# 3. Gộp tất cả các embedding và metadata
# --------------------
print("[+] Merging all embeddings and metadata...")

all_embeddings = []
all_metadata = []

for fname in sorted(os.listdir(output_dir)):
    if fname.startswith("embeddings_part_") and fname.endswith(".npy"):
        emb = np.load(os.path.join(output_dir, fname))
        all_embeddings.append(emb)

    if fname.startswith("metadata_part_") and fname.endswith(".json"):
        with open(os.path.join(output_dir, fname), "r", encoding="utf-8") as f:
            metadata = json.load(f)
            all_metadata.extend(metadata)

all_embeddings = np.vstack(all_embeddings)

d = all_embeddings.shape[1]
index = faiss.IndexFlatIP(d)
index.add(all_embeddings)

faiss.write_index(index, "wiki_chunks.index")
with open("wiki_metadata.json", "w", encoding="utf-8") as f:
    json.dump(all_metadata, f, ensure_ascii=False, indent=2)

print(f"[✓] Done! Total vectors: {len(all_embeddings)}")

[+] Merging all embeddings and metadata...
[✓] Done! Total vectors: 93996
